In [1]:
import pandas as pd
import sys
import time
from difflib import SequenceMatcher
from tqdm import tqdm
import time
from multiprocessing import Pool, cpu_count
import numpy as np

sys.path.append('../functions')
from hash_functions import normalize_text_for_hash


# Cargar el archivo generado en el paso anterior (hash_creation.ipynb)
df = pd.read_csv('../../data/03_extracted/regulation_mentions_extracted.csv')
df = df[~df['art_id'].str.contains('_TRANS', na=False)]
df.head()

,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count
2,37159EFC,37159EFC_19,Reglamento de esta Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,Federal que haya sido solicitado por un produc...,30,30
4,3A2C8488,3A2C8488_168,Reglamento de la presente Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,Consejo de Resiliencia tendrá órganos de traba...,30,1
7,C57B66D6,C57B66D6_92,Reglamento de esta ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,", dentro del plazo a que se refiere el párrafo...",30,30
8,C57B66D6,C57B66D6_100,Reglamento de esta Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,Vicies . En lo no previsto en esta sección y t...,28,10
10,6DAE5BFB,6DAE5BFB_20,Reglamento de esta Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,que se integrará por un representante de cada ...,30,30


In [2]:
ley_reglamentos = df[df['entity_label'].isin(['LEY_Y_REGLAMENTOS','LEY_CON_REGLAMENTOS'])]

ley_reglamentos['entity_text'].unique()



array(['Reglamento de esta Ley', 'Reglamento de la presente Ley',
       'Reglamento de esta ley', 'esta ley y sus reglamentos',
       'reglamento de la presente Ley', 'reglamento de esta Ley',
       'esta Ley y sus reglamentos', 'reglamento de esta ley',
       'reglamento de la presente ley', 'esta Ley y sus Reglamentos',
       'esta Ley y con el reglamento', 'Reglamento de la presente ley'],
      dtype=object)

In [3]:
# Leer el archivo con el nombre de las leyes completas 
leyes_completas = pd.read_csv('../../data/02_catalogs/leyes_hash.csv')
leyes_completas.head()



,nombre,fecha_publicacion,fecha_actualizacion,link_pdf,link_docx,source_url,gov_level,nombre_normalized,doc_id
0,Constitucion Politica de la Ciudad de Mexico,05/02/17,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,CONSTITUCIONPOLITICADELACIUDADDEMEXICO,234F69A3
1,Ley de Proteccion de Datos Personales en Poses...,03/10/08,NaN,https://data.consejeria.cdmx.gob.mx//images/le...,NaN,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYDEPROTECCIONDEDATOSPERSONALESENPOSESIONDESU...,80360428
2,Ley de Acceso de las Mujeres a una Vida Libre ...,25/06/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYDEACCESODELASMUJERESAUNAVIDALIBREDEVIOLENCI...,74FF23BF
3,Ley de los Derechos de Ninas Ninos y Adolescen...,02/04/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYDELOSDERECHOSDENINASNINOSYADOLESCENTESDELAC...,FB6CCCAE
4,Ley para el Reconocimiento y la Atencion de la...,31/03/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYPARAELRECONOCIMIENTOYLAATENCIONDELASPERSONA...,6C800A8E


In [4]:
# Hacer un left join de el nombre y el tipo de ley a las menciones actuales. 
# De esta manera se obtiene el nombre de la ley de cada uno de los acrtículos 
# Entonces su puede saber de que ley y de que reglamenot de está hablando. 

ley_reglamentos = ley_reglamentos.join(
    leyes_completas[['doc_id', 'nombre']].set_index('doc_id'), 
    on='doc_id', 
    how='left'
)

# Crear el nombre del reglamento al que hace referencia la mención
ley_reglamentos['nombre_reglamento'] =  'Reglamento de la' + ' ' +ley_reglamentos['nombre']


ley_reglamentos['nombre_reglamento_normalized'] = ley_reglamentos['nombre_reglamento'].apply(normalize_text_for_hash)

ley_reglamentos.shape



(217, 11)

In [5]:
ley_reglamentos = ley_reglamentos[~ley_reglamentos['nombre'].str.startswith('Regla', na=False)]
ley_reglamentos.shape

(215, 11)

In [6]:
ley_reglamentos.shape
ley_reglamentos

,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count,nombre,nombre_reglamento,nombre_reglamento_normalized
2,37159EFC,37159EFC_19,Reglamento de esta Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,Federal que haya sido solicitado por un produc...,30,30,Ley de Filmaciones de la Ciudad de Mexico,Reglamento de la Ley de Filmaciones de la Ciud...,REGLAMENTODELALEYDEFILMACIONESDELACIUDADDEMEXICO
4,3A2C8488,3A2C8488_168,Reglamento de la presente Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,Consejo de Resiliencia tendrá órganos de traba...,30,1,Ley de Gestion Integral de Riesgos y Proteccio...,Reglamento de la Ley de Gestion Integral de Ri...,REGLAMENTODELALEYDEGESTIONINTEGRALDERIESGOSYPR...
7,C57B66D6,C57B66D6_92,Reglamento de esta ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,", dentro del plazo a que se refiere el párrafo...",30,30,Ley del Notariado para la Ciudad de Mexico,Reglamento de la Ley del Notariado para la Ciu...,REGLAMENTODELALEYDELNOTARIADOPARALACIUDADDEMEXICO
8,C57B66D6,C57B66D6_100,Reglamento de esta Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,Vicies . En lo no previsto en esta sección y t...,28,10,Ley del Notariado para la Ciudad de Mexico,Reglamento de la Ley del Notariado para la Ciu...,REGLAMENTODELALEYDELNOTARIADOPARALACIUDADDEMEXICO
10,6DAE5BFB,6DAE5BFB_20,Reglamento de esta Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,que se integrará por un representante de cada ...,30,30,Ley de Adquisiciones para el Distrito Federal,Reglamento de la Ley de Adquisiciones para el ...,REGLAMENTODELALEYDEADQUISICIONESPARAELDISTRITO...
...,...,...,...,...,...,...,...,...,...,...,...
286,AF96B37C,AF96B37C_129,Reglamento de la presente Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,El tratamiento de personas con consumo de sust...,22,13,Ley de Salud de la Ciudad De Mexico,Reglamento de la Ley de Salud de la Ciudad De ...,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO
287,AF96B37C,AF96B37C_131,Reglamento de la presente Ley,LEY_Y_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,Ciudad de México que presten servicios de aten...,30,1,Ley de Salud de la Ciudad De Mexico,Reglamento de la Ley de Salud de la Ciudad De ...,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO
288,AF96B37C,AF96B37C_159,esta ley y sus reglamentos,LEY_CON_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,le correspondan al Gobierno ; Hacer del conoci...,30,30,Ley de Salud de la Ciudad De Mexico,Reglamento de la Ley de Salud de la Ciudad De ...,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO
289,AF96B37C,AF96B37C_159,esta ley y sus reglamentos,LEY_CON_REGLAMENTOS,SPECIFIC_LAW_MENTIONS,le correspondan al Gobierno ; Hacer del conoci...,30,30,Ley de Salud de la Ciudad De Mexico,Reglamento de la Ley de Salud de la Ciudad De ...,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO


In [7]:
# Leer la tabla de leyes completas
leyes_completas = pd.read_csv('../../data/02_catalogs/leyes_hash.csv')
leyes_completas = leyes_completas[leyes_completas['nombre_normalized'].str.startswith('REGLA', na=False)]


leyes_completas.head()

,nombre,fecha_publicacion,fecha_actualizacion,link_pdf,link_docx,source_url,gov_level,nombre_normalized,doc_id
181,Reglamento de la Ley de Turismo de la Ciudad d...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDETURISMODELACIUDADDEMEXICO,FAA2EE90
182,Reglamento del Registro Civil de la Ciudad de ...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELREGISTROCIVILDELACIUDADDEMEXICO,4198B460
183,Reglamento de la Ley de Establecimientos Merca...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,NaN,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDEESTABLECIMIENTOSMERCANTILES...,7566EC96
184,Reglamento de la Ley de Cultura Civica de la C...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,NaN,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDECULTURACIVICADELACIUDADDEME...,C6E9B915
185,Reglamento de la Ley del Notariado en la Ciuda...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDELNOTARIADOENLACIUDADDEMEXICO,D2CE6E11


In [8]:
def match_official_docs(oficial_docs_df, leyes_completas_df, threshold=0.8):
    """
    Matching entre entity_text y name con estrategia de fallback.
    Si el match original no es > 0.9, intenta con 'ciudad de mexico' -> 'distrito federal'.
    Con timer y barra de progreso.
    """
    start_time = time.time()
    results = []
    
    print(f"Iniciando matching de {len(oficial_docs_df):,} documentos oficiales contra {len(leyes_completas_df):,} referencias...")
    print(f"Threshold: {threshold}")
    print(f"Total de comparaciones: {len(oficial_docs_df) * len(leyes_completas_df):,}")
    
    # Agregar barra de progreso
    for idx, row in tqdm(oficial_docs_df.iterrows(), total=len(oficial_docs_df), desc="Procesando"):
        entity_text_original = row['nombre_reglamento_normalized']
        art_id = row['art_id']
        
        # Crear versión alternativa reemplazando "CIUDADDEMEXICO" por "DISTRITOFEDERAL"
        entity_text_created = entity_text_original.replace('CIUDADDEMEXICO', 'DISTRITOFEDERAL')
        
        best_match = None
        best_match_name = None
        best_score = 0
        match_version = ''
        
        # PRIMER INTENTO: Buscar el mejor match con el nombre original
        for _, law_row in leyes_completas_df.iterrows():
            law_name = law_row['nombre_normalized']
            doc_id = law_row['doc_id']
            
            # Calcular similitud
            similarity = SequenceMatcher(None, entity_text_original.lower(), law_name.lower()).ratio()
            
            if similarity > best_score and similarity >= threshold:
                best_score = similarity
                best_match = doc_id
                best_match_name = law_row['nombre']
                match_version = 'original'
        
        # SEGUNDO INTENTO: Si el match no es > 0.9 y hay una versión alternativa diferente, intentar con ella
        if best_score <= 0.9 and entity_text_created != entity_text_original:
            for _, law_row in leyes_completas_df.iterrows():
                law_name = law_row['nombre_normalized']
                doc_id = law_row['doc_id']
                
                # Calcular similitud con la versión alternativa
                similarity = SequenceMatcher(None, entity_text_created.lower(), law_name.lower()).ratio()
                
                if similarity > best_score and similarity >= threshold:
                    best_score = similarity
                    best_match = doc_id
                    best_match_name = law_row['nombre']
                    match_version = 'created'
        
        # Agregar resultado
        results.append({
            'art_id': art_id,
            'nombre_reglamento_normalized': entity_text_original,
            'nombre_reglamento_normalized_created': entity_text_created,
            'match_version_used': match_version,
            'doc_id': best_match if best_match else '',
            'nombre_ley_matched': best_match_name if best_match_name else '',
            'similarity': best_score if best_match else 0
        })
    
    elapsed_time = time.time() - start_time
    print(f"\nMatching completado en {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)")
    print(f"Matches encontrados: {sum(1 for r in results if r['doc_id'])}")
    print(f"Matches con versión original: {sum(1 for r in results if r['match_version_used'] == 'original')}")
    print(f"Matches con versión creada (distrito federal): {sum(1 for r in results if r['match_version_used'] == 'created')}")
    
    return pd.DataFrame(results)

In [9]:
results = match_official_docs(ley_reglamentos, leyes_completas, threshold=0.9)
results 

Iniciando matching de 215 documentos oficiales contra 291 referencias...
Threshold: 0.9
Total de comparaciones: 62,565


Procesando:   0%|          | 0/215 [00:00<?, ?it/s]

Procesando: 100%|██████████| 215/215 [00:13<00:00, 16.15it/s]


Matching completado en 13.32 segundos (0.22 minutos)
Matches encontrados: 184
Matches con versión original: 112
Matches con versión creada (distrito federal): 72


,art_id,nombre_reglamento_normalized,nombre_reglamento_normalized_created,match_version_used,doc_id,nombre_ley_matched,similarity
0,37159EFC_19,REGLAMENTODELALEYDEFILMACIONESDELACIUDADDEMEXICO,REGLAMENTODELALEYDEFILMACIONESDELADISTRITOFEDERAL,created,807464C1,Reglamento de la Ley de Filmaciones del Distri...,0.989691
1,3A2C8488_168,REGLAMENTODELALEYDEGESTIONINTEGRALDERIESGOSYPR...,REGLAMENTODELALEYDEGESTIONINTEGRALDERIESGOSYPR...,original,D61F4C69,Reglamento de la Ley de Gestion Integral de Ri...,1.000000
2,C57B66D6_92,REGLAMENTODELALEYDELNOTARIADOPARALACIUDADDEMEXICO,REGLAMENTODELALEYDELNOTARIADOPARALADISTRITOFED...,original,D2CE6E11,Reglamento de la Ley del Notariado en la Ciuda...,0.937500
3,C57B66D6_100,REGLAMENTODELALEYDELNOTARIADOPARALACIUDADDEMEXICO,REGLAMENTODELALEYDELNOTARIADOPARALADISTRITOFED...,original,D2CE6E11,Reglamento de la Ley del Notariado en la Ciuda...,0.937500
4,6DAE5BFB_20,REGLAMENTODELALEYDEADQUISICIONESPARAELDISTRITO...,REGLAMENTODELALEYDEADQUISICIONESPARAELDISTRITO...,original,47C1641A,Reglamento de la Ley de Adquisiciones para el ...,1.000000
...,...,...,...,...,...,...,...
210,AF96B37C_129,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO,REGLAMENTODELALEYDESALUDDELADISTRITOFEDERAL,created,87EDE287,Reglamento de la Ley de Salud del Distrito Fed...,0.988235
211,AF96B37C_131,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO,REGLAMENTODELALEYDESALUDDELADISTRITOFEDERAL,created,87EDE287,Reglamento de la Ley de Salud del Distrito Fed...,0.988235
212,AF96B37C_159,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO,REGLAMENTODELALEYDESALUDDELADISTRITOFEDERAL,created,87EDE287,Reglamento de la Ley de Salud del Distrito Fed...,0.988235
213,AF96B37C_159,REGLAMENTODELALEYDESALUDDELACIUDADDEMEXICO,REGLAMENTODELALEYDESALUDDELADISTRITOFEDERAL,created,87EDE287,Reglamento de la Ley de Salud del Distrito Fed...,0.988235


In [10]:
results[(results['similarity'] < 0.95) & (results['similarity'] > 0) & (~results['art_id'].str.contains('_TRANS', na=False))]


,art_id,nombre_reglamento_normalized,nombre_reglamento_normalized_created,match_version_used,doc_id,nombre_ley_matched,similarity
2,C57B66D6_92,REGLAMENTODELALEYDELNOTARIADOPARALACIUDADDEMEXICO,REGLAMENTODELALEYDELNOTARIADOPARALADISTRITOFED...,original,D2CE6E11,Reglamento de la Ley del Notariado en la Ciuda...,0.937500
3,C57B66D6_100,REGLAMENTODELALEYDELNOTARIADOPARALACIUDADDEMEXICO,REGLAMENTODELALEYDELNOTARIADOPARALADISTRITOFED...,original,D2CE6E11,Reglamento de la Ley del Notariado en la Ciuda...,0.937500
19,D0D23E5F_33,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752
20,D0D23E5F_34,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752
21,D0D23E5F_35,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752
22,D0D23E5F_46,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752
23,D0D23E5F_48,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752
24,D0D23E5F_50,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752
25,D0D23E5F_63,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752
26,D0D23E5F_65,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,REGLAMENTODELALEYORGANICADELAFISCALIAGENERALDE...,created,55FBF591,Reglamento de la Ley Organica de la Procuradur...,0.912752


In [11]:
results[(results['similarity'] == 0) & (~results['art_id'].str.contains('_TRANS', na=False))]['nombre_reglamento_normalized'].unique()


array(['REGLAMENTODELALEYDELREGIMENPATRIMONIALYDELSERVICIOPUBLICO',
       'REGLAMENTODELALEYDELOSDERECHOSCULTURALESDELOSHABITANTESYVISITANTESDELACIUDADDEMEXICO',
       'REGLAMENTODELALEYQUEESTABLECEELDERECHOARECIBIRUNAPOYOALIMENTARIOALASMADRESSOLASDEESCASOSRECURSOSRESIDENTESENLACIUDADDEMEXICO',
       'REGLAMENTODELALEYDEPLANEACIONDEMOGRAFICAYESTADISTICAPARALAPOBLACIONDELDISTRITOFEDERAL',
       'REGLAMENTODELALEYPARALAPROTECCIONINTEGRALDEPERSONASDEFENSORASDEDERECHOSHUMANOSYPERIODISTASDELDISTRITOFEDERAL',
       'REGLAMENTODELALEYDELADEFENSORIAPUBLICADELDISTRITOFEDERAL',
       'REGLAMENTODELALEYQUEREGULAELUSODETECNOLOGIAPARALASEGURIDADPUBLICADELDISTRITOFEDERAL',
       'REGLAMENTODELALEYDEBEBESEGURODELACIUDADDEMEXICO',
       'REGLAMENTODELALEYDEFOMENTOPARALALECTURAYELLIBRODELACIUDADDEMEXICO',
       'REGLAMENTODELALEYDELINSTITUTODEVERIFICACIONADMINISTRATIVADELACIUDADDEMEXICO',
       'REGLAMENTODELALEYDECIUDADANIADIGITALDELACIUDADDEMEXICO',
       'REGLAMENTODELALEYPARAELRECONOCIMI

In [12]:
# Ninguno de estos reglamentos que no fueron matcheados existe.

In [13]:
results['type_of_relation'] = 'mencion_reglamento'
results.to_csv('../../data/04_matched/regulation_mentions_matched.csv', index=False)

Además de la mención al reglamento existe un mención al mismo documento 

In [14]:
ley_reglamentos = df[df['entity_label'].isin(['LEY_Y_REGLAMENTOS','LEY_CON_REGLAMENTOS'])]
self_references_reglamentos = ley_reglamentos[['art_id','doc_id']]

self_references_reglamentos['type_of_relation'] = 'mencion_reglamento'

#self_references_reglamentos
self_references_reglamentos.to_csv('../../data/04_matched/regulation_mentions_matched_self_references.csv', index=False)


/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_62514/524967706.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self_references_reglamentos['type_of_relation'] = 'mencion_reglamento'
